# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [9]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [11]:
# Verificando se existe uma coluna relacionada à idade do pai no conjunto de dados
nomes_colunas = sinasc_raw.columns.tolist()
colunas_idade_pai = [col for col in nomes_colunas if 'PAI' in col.upper() and 'IDADE' in col.upper()]
colunas_idade_pai



['IDADEPAI']

In [13]:
# Calcular a idade média das mães e dos pais por município
idades_medias = sinasc_raw.groupby('munResNome').agg(
    Idade_Média_Mães=pd.NamedAgg(column='IDADEMAE', aggfunc='mean'),
    Idade_Média_Pais=pd.NamedAgg(column='IDADEPAI', aggfunc='mean')
).reset_index()

# Exibir as primeiras linhas do DataFrame resultante
idades_medias.head()


,munResNome,Idade_Média_Mães,Idade_Média_Pais
0,Alta Floresta D'Oeste,25.991826,29.548701
1,Alto Alegre dos Parecis,24.844156,29.179104
2,Alto Paraíso,24.960177,28.833333
3,Alvorada D'Oeste,25.770732,30.757282
4,Ariquemes,25.607866,32.466667


### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 07/06, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [16]:
# Primeiro, vamos verificar as colunas disponíveis para encontrar as relacionadas a data de nascimento, sexo, peso e escolaridade da mãe
sinasc_raw.columns.tolist()


['ORIGEM',
 'CODESTAB',
 'CODMUNNASC',
 'LOCNASC',
 'IDADEMAE',
 'ESTCIVMAE',
 'ESCMAE',
 'CODOCUPMAE',
 'QTDFILVIVO',
 'QTDFILMORT',
 'CODMUNRES',
 'GESTACAO',
 'GRAVIDEZ',
 'PARTO',
 'CONSULTAS',
 'DTNASC',
 'HORANASC',
 'SEXO',
 'APGAR1',
 'APGAR5',
 'RACACOR',
 'PESO',
 'IDANOMAL',
 'DTCADASTRO',
 'CODANOMAL',
 'NUMEROLOTE',
 'VERSAOSIST',
 'DTRECEBIM',
 'DIFDATA',
 'DTRECORIGA',
 'NATURALMAE',
 'CODMUNNATU',
 'CODUFNATU',
 'ESCMAE2010',
 'SERIESCMAE',
 'DTNASCMAE',
 'RACACORMAE',
 'QTDGESTANT',
 'QTDPARTNOR',
 'QTDPARTCES',
 'IDADEPAI',
 'DTULTMENST',
 'SEMAGESTAC',
 'TPMETESTIM',
 'CONSPRENAT',
 'MESPRENAT',
 'TPAPRESENT',
 'STTRABPART',
 'STCESPARTO',
 'TPNASCASSI',
 'TPFUNCRESP',
 'TPDOCRESP',
 'DTDECLARAC',
 'ESCMAEAGR1',
 'STDNEPIDEM',
 'STDNNOVA',
 'CODPAISRES',
 'TPROBSON',
 'PARIDADE',
 'KOTELCHUCK',
 'CONTADOR',
 'munResStatus',
 'munResTipo',
 'munResNome',
 'munResUf',
 'munResLat',
 'munResLon',
 'munResAlt',
 'munResArea']

In [18]:
# Filtrando o conjunto de dados para bebês nascidos em 07 de junho e calculando o peso médio por sexo e faixa de escolaridade da mãe
sinasc_07_06 = sinasc_raw[sinasc_raw['DTNASC'].str.startswith('2019-06-07')]

# Agrupar por sexo e escolaridade da mãe, calculando o peso médio
peso_medio_bebes = sinasc_07_06.groupby(['SEXO', 'ESCMAE']).agg(
    Peso_Médio=pd.NamedAgg(column='PESO', aggfunc='mean')
).reset_index()

peso_medio_bebes


,SEXO,ESCMAE,Peso_Médio
0,Feminino,12 anos ou mais,3360.571429
1,Feminino,4 a 7 anos,3410.000000
2,Feminino,8 a 11 anos,3044.666667
3,Feminino,Nenhuma,3845.000000
4,Masculino,12 anos ou mais,3463.750000
5,Masculino,4 a 7 anos,2869.250000
6,Masculino,8 a 11 anos,3278.800000
7,Masculino,Nenhuma,350.000000


### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [20]:
# Calculando o município com o menor número de nascimentos em 2019
nascimentos_por_municipio = sinasc_raw['munResNome'].value_counts().idxmin()

# Filtrando o conjunto de dados para esse município específico
dados_municipio_menor_nasc = sinasc_raw[sinasc_raw['munResNome'] == nascimentos_por_municipio]

# Calculando a idade média, máxima e mínima das mães e dos pais nesse município
idade_maes = dados_municipio_menor_nasc['IDADEMAE'].agg(['mean', 'max', 'min'])
idade_pais = dados_municipio_menor_nasc['IDADEPAI'].agg(['mean', 'max', 'min'])

nascimentos_por_municipio, idade_maes, idade_pais


('Município ignorado - RO',
 mean    24.0
 max     24.0
 min     24.0
 Name: IDADEMAE, dtype: float64,
 mean    22.0
 max     22.0
 min     22.0
 Name: IDADEPAI, dtype: float64)

### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



In [22]:
# Convertendo a coluna 'DTNASC' para o formato datetime e filtrando os nascimentos de março de 2019
sinasc_raw['DTNASC'] = pd.to_datetime(sinasc_raw['DTNASC'])
nascimentos_marco = sinasc_raw[sinasc_raw['DTNASC'].dt.month == 3]

# Identificando o município com o maior número de nascimentos em março
municipio_mais_nasc_marco = nascimentos_marco['munResNome'].value_counts().idxmax()

# Filtrando os dados para esse município
dados_municipio_mais_nasc_marco = nascimentos_marco[nascimentos_marco['munResNome'] == municipio_mais_nasc_marco]

# Calculando a quantidade de filhos vivos média, máxima e mínima nesse município
qtde_filhos_vivos = dados_municipio_mais_nasc_marco['QTDFILVIVO'].agg(['mean', 'max', 'min'])

# Calculando a idade média, máxima e mínima dos pais nesse município
idade_pais_marco = dados_municipio_mais_nasc_marco['IDADEPAI'].agg(['mean', 'max', 'min'])

municipio_mais_nasc_marco, qtde_filhos_vivos, idade_pais_marco


('Porto Velho',
 mean    1.295056
 max     9.000000
 min     0.000000
 Name: QTDFILVIVO, dtype: float64,
 mean    34.629032
 max     62.000000
 min     19.000000
 Name: IDADEPAI, dtype: float64)

### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

Exemplo:
- Ah, descobri que a idade mediana das mulheres que deram a luz no ano de 2019 dos municipios x é maior que y.

In [6]:
sinasc_raw.dtypes

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResUf       object
munResLat     float64
munResLon     float64
munResAlt     float64
munResArea    float64
Length: 69, dtype: object

In [7]:
sinasc_raw.select_dtypes(exclude=['int64','float64'])

,ESCMAE,GESTACAO,GRAVIDEZ,PARTO,DTNASC,SEXO,RACACOR,DTCADASTRO,CODANOMAL,VERSAOSIST,DTRECEBIM,munResStatus,munResTipo,munResNome,munResUf
0,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-19,Masculino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
1,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-21,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
2,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-25,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
3,12 anos ou mais,37 a 41 semanas,Única,Cesáreo,2019-03-20,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia
4,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-03-23,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,12 anos ou mais,32 a 36 semanas,Única,Cesáreo,2019-12-13,Masculino,Branca,2020-09-28,NaN,3.2.01,2020-10-13,ATIVO,MUNIC,Vilhena,Rondônia
27024,8 a 11 anos,37 a 41 semanas,Única,Vaginal,2019-10-04,Masculino,Branca,2019-10-16,NaN,3.2.01,2019-10-21,ATIVO,MUNIC,Chupinguaia,Rondônia
27025,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-08-02,Masculino,Parda,2019-08-07,NaN,3.2.00,2019-08-07,ATIVO,MUNIC,Vilhena,Rondônia
27026,8 a 11 anos,32 a 36 semanas,Única,Vaginal,2019-12-23,Masculino,Parda,2020-01-07,NaN,3.2.00,2020-01-07,ATIVO,MUNIC,Vilhena,Rondônia


In [4]:
sinasc_raw.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

### Distribuição de Nascimentos por Município:
    -Porto Velho destacou-se com o maior número de nascimentos em março, o que é esperado, dado que é a capital de Rondônia e uma das cidades mais populosas do estado. Este padrão de nascimentos concentrados nas capitais ou grandes cidades é comum e reflete a centralização de recursos médicos e de saúde.
### Idade dos Pais e Mães:
    -A idade média dos pais em Porto Velho (34,63 anos) é significativamente mais alta comparada à idade dos pais no município com menos nascimentos (22 anos), o que pode indicar uma tendência de pais mais velhos em áreas urbanas maiores ou mais desenvolvidas. Isso pode estar relacionado a fatores como carreira profissional e estabilidade financeira antes de ter filhos.